# Regularly Oversimplifying Neural Networks

## Background

What we want to do is be able to look at two neural networks and find similar features between them. This requires a data format that makes this easy.

### Computational Classes

Let's talk about different ways neural networks could be represented.

#### Turing Machines

PCs are subsets of Turing machines. They're not quite Turing machines, because they don't have infinite memory. Neural networks, however, are not Turing-complete. If they were, we would often see models that never halt, and as [Alan Turing pointed out](https://en.wikipedia.org/wiki/Halting_problem), a computer can't look at just any Turing machine know whether or not it halts. So this would be a superset of neural network models.

#### Context-Free Languages

Most programming languages can be syntactically parsed by just using context-free languages. You can imagine it as a graph we have to traverse through. This can still end up in an infinite loop, like in:

```
prog  -> block
block -> prog
```

but it's easier to verify that this doesn't happen.

#### Regular Language

This is quite possibly the worst representation for a neural network. It's not that it can't be done. Assembly language is actually a regular language. But semantic analysis is needed in order to make that work. Regular languages can't be recursive, so any high-level language with nested parentheses (like, for example, all of them), don't qualify.

## The Plan

The plan is to make a representation of a neural network that allows us to easily find similarities between multiple of them.

So, we're going to do what we just said is the worst way to represent a neural network. We're going to turn a neural network into a regular language, making it difficult to represent connections. But, if we ignore the connections, we can avoid subgraph isomorphism. It's not quite as simple as a substring search, but it's similar.

This is definitely not impossible. A single neuron can be represented as a regular expression. A layer is just a list of neurons, and a neural network is just a list of layers. We can't represent connections without doing semantic analysis, but since we're trying to avoid subgraph isomorphism anyway, this is fine.

Obviously, this won't be perfect. We don't imagine this could work as an alternative to subgraph isomorphism, but when there are 784 input nodes, we need faster ways to do this. This can be used as a pruning technique to narrow down which patterns are appearing in the neural network, so not everything needs to be checked.

## Creating a Neural Network

This neural network was created using the [PyTorch Quickstart Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html). Although, the number of inputs and outputs is different in the second layer, for reasons that will be explained later.

In [ ]:
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, IterableDataset
from torch.optim import SGD
from torchvision import datasets
from torchvision.transforms import ToTensor

BATCH_SIZE = 64
EPOCHS = 3
WEIGHTS = [28 * 28, 256, 16, 10]

# define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(WEIGHTS[0], WEIGHTS[1]),
            nn.ReLU(),
            nn.Linear(WEIGHTS[1], WEIGHTS[2]),
            nn.ReLU(),
            nn.Linear(WEIGHTS[2], WEIGHTS[3])
        )
    
    def forward(self, x) -> Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss, optimizer: SGD):
    size = len(dataloader.dataset) # type: ignore
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print batch information
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss):
    size = len(dataloader.dataset) # type: ignore
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print("Test Error:")
    print(f"\tAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    print()

# get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

# download training data from open datasets
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

# download test data from open datasets
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

# create data loaders
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# train model
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Using cuda


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Epoch 1
-------------------------------
loss: 2.352911  [    0/60000]
loss: 2.321046  [ 6400/60000]
loss: 2.302899  [12800/60000]
loss: 2.284874  [19200/60000]
loss: 2.270969  [25600/60000]
loss: 2.273806  [32000/60000]
loss: 2.282810  [38400/60000]
loss: 2.264515  [44800/60000]
loss: 2.258900  [51200/60000]
loss: 2.261303  [57600/60000]
Test Error:
	Accuracy: 20.2%, Avg loss: 2.233701

Epoch 2
-------------------------------
loss: 2.249836  [    0/60000]
loss: 2.235572  [ 6400/60000]
loss: 2.193583  [12800/60000]
loss: 2.208789  [19200/60000]
loss: 2.169065  [25600/60000]
loss: 2.167217  [32000/60000]
loss: 2.182488  [38400/60000]
loss: 2.130972  [44800/60000]
loss: 2.117302  [51200/60000]
loss: 2.155035  [57600/60000]
Test Error:
	Accuracy: 31.2%, Avg loss: 2.100964

Epoch 3
-------------------------------
loss: 2.102561  [    0/60000]
loss: 2.095799  [ 6400/60000]
loss: 2.013748  [12800/60000]
loss:

## Getting Data Out of the Neural Network

### Getting Information out of PyTorch

As shown below, each layer of a neural network is represented using two Tensors. Tensors are thankfully iterable, and we can get the complete information for a neuron by getting its weights, and its bias.

In [ ]:
from typing import OrderedDict

state: OrderedDict[str, Tensor] = model.state_dict() # type: ignore
for key in state.keys():
    print(type(key), key, type(key), len(state[key]))

for n in range(5):
    print(f"\nNeuron {n}:")
    print("Weights: ", str(state["linear_relu_stack.0.weight"][n][0:5]), "...")
    print("Bias:    ", state["linear_relu_stack.0.bias"][n])

<class 'str'> linear_relu_stack.0.weight <class 'str'> 256
<class 'str'> linear_relu_stack.0.bias <class 'str'> 256
<class 'str'> linear_relu_stack.2.weight <class 'str'> 16
<class 'str'> linear_relu_stack.2.bias <class 'str'> 16
<class 'str'> linear_relu_stack.4.weight <class 'str'> 10
<class 'str'> linear_relu_stack.4.bias <class 'str'> 10

Neuron 0:
Weights:  tensor([ 0.0147, -0.0318,  0.0217,  0.0187, -0.0346], device='cuda:0') ...
Bias:     tensor(0.0103, device='cuda:0')

Neuron 1:
Weights:  tensor([-0.0343,  0.0297, -0.0155,  0.0143, -0.0098], device='cuda:0') ...
Bias:     tensor(0.0366, device='cuda:0')

Neuron 2:
Weights:  tensor([ 0.0128, -0.0255, -0.0091,  0.0221, -0.0174], device='cuda:0') ...
Bias:     tensor(-0.0117, device='cuda:0')

Neuron 3:
Weights:  tensor([-0.0335,  0.0033,  0.0044, -0.0114, -0.0136], device='cuda:0') ...
Bias:     tensor(-0.0074, device='cuda:0')

Neuron 4:
Weights:  tensor([-0.0217, -0.0119, -0.0351, -0.0123,  0.0001], device='cuda:0') ...
Bias: 

### Structuring the Data

A neural network can be (poorly) represented with a list of layers. A layer can be represented with a list of neurons. A neuron can be represented by a bias and a list of weights.

#### Equivalent Neurons

But, we still need to make sure that equivalent neurons are equivalent. We'll consider two neurons to be equivalent if the same inputs produce the same output. Take the following two neurons:

```
Input | Neuron 1 Output | Neuron 2 Output
-----------------------------------------
 0, 0 |        0        |        0
 1, 0 |        0        |        0
 1, 1 |        1        |        1
 0, 1 |        0        |        0
```

Those two neurons are equivalent, because the same input results in the same output. However, the following two neurons would not be equivalent:

```
Input | Neuron 1 Output | Neuron 2 Output
-----------------------------------------
 0, 0 |        0        |        1
 1, 0 |        0        |        0
 1, 1 |        1        |        1
 0, 1 |        0        |        0
```

Part of the work for this is to sort the weights before putting them in the list. This will be helpful so that we don't have to do any isomorphism to have like inputs. We haven't rigorously proven this, but it seems like sorting the weights should not break the equivalency check. If you can show otherwise, please tell us.

There is one more problem, which is that each neuron in the first layer has 784 weights. That means there are `2 ** 784` possible inputs, which we cannot possibly run completely on any computer in the world. Small neural networks, however, can run through all of the possibilities. For the sake of this hackathon, we decreased the size of the neural network to make it feasible. Our plan is to discuss possible optimizations later.

In [ ]:
from typing import List
import itertools

next_neuron_id = 0

class NeuronModel:
    def __init__(self, weights, bias):
        global next_neuron_id
        self.id = next_neuron_id
        self.bias = float(bias)
        self.weights = sorted([float(weight) for weight in weights])
        next_neuron_id += 1

    def __str__(self) -> str:
        string = ""
        string += f"\nNeuron {self.id}:"
        string += "\n\tWeight #: " + str(len(self.weights))
        string += "\n\tWeights : " + str(self.weights[0:2]) + "..."
        string += "\n\tBias    : " + str(self.bias)
        string += "\n\tStates  : " + str(len(self.excited_states()))
        return string

    def __eq__(self, other) -> bool:
        # optimization: don't bother if the neurons have different weight counts
        if len(self.weights) != len(other.weights): return False
        return self.excited_states() == other.excited_states()

    def excited_states(self) -> List[List[int]]:
        states = []

        if sum(self.weights) > 0: states.append(list(range(len(self.weights))))

        for i in range(len(self.weights)):
            for state in itertools.combinations(range(len(self.weights)), i):
                total = 0
                for w in state:
                    total += float(self.weights[w])
                if total + self.bias > 0: states.append(list(state))

        return states

state_layer_weights = state["linear_relu_stack.4.weight"]
state_layer_biases = state["linear_relu_stack.4.bias"]
for n in range(int(len(state_layer_weights) / 2)):
    neuron_weights = state_layer_weights[n]
    neuron_bias = state_layer_biases[n]
    neuron = NeuronModel(neuron_weights, neuron_bias)
    print(neuron)
    print()

and_neuron = NeuronModel([0.7, 0.7], -1.0)
and_neuron2 = NeuronModel([0.6, 0.6], -1.0)
print(and_neuron == and_neuron2)
or_neuron = NeuronModel([1.0, 1.0], 0.0)
print(and_neuron == or_neuron)


Neuron 0:
	Weight #: 16
	Weights : [-0.23089340329170227, -0.22210676968097687]...
	Bias    : -0.125585675239563
	States  : 41115


Neuron 1:
	Weight #: 16
	Weights : [-0.2764178216457367, -0.22455595433712006]...
	Bias    : -0.022253355011343956
	States  : 39634


Neuron 2:
	Weight #: 16
	Weights : [-0.23412220180034637, -0.20991556346416473]...
	Bias    : -0.12217739224433899
	States  : 18417


Neuron 3:
	Weight #: 16
	Weights : [-0.3188047409057617, -0.22931914031505585]...
	Bias    : -0.24843735992908478
	States  : 40905


Neuron 4:
	Weight #: 16
	Weights : [-0.2463586926460266, -0.21394309401512146]...
	Bias    : -0.008997483178973198
	States  : 34440

True
False


Now, we'll implement a `LayerModel`.

In [ ]:
class LayerModel:
    def __init__(self, weights: Tensor, biases: Tensor):
        self.neurons = [NeuronModel(weights[i], biases[i]) for i in range(len(weights))]

    def __len__(self):
        return len(self.neurons)

    def __iter__(self):
        return iter(self.neurons)

    def __getitem__(self, i: int) -> NeuronModel:
        return self.neurons[i]

    def is_superlayer_of(self, other) -> bool:
        for neuron in other:
            if neuron not in self: return False
        return True

last_layer = LayerModel(state_layer_weights, state_layer_biases)
dumblayer = LayerModel([[0.9, -0.8, 0.7], [-0.6, 0.5, -0.4]], [0.5, -0.5]) # type: ignore
sub_weights = list(state_layer_weights[2:5]) + list(state_layer_weights[7:9])
sub_biases = list(state_layer_biases[2:5]) + list(state_layer_biases[7:9])
sublayer = LayerModel(sub_weights, sub_biases) # type: ignore
print(last_layer.is_superlayer_of(dumblayer))
print(last_layer.is_superlayer_of(sublayer))

False
True


The rest of the neural network is just as trivial.

In [ ]:
class NNModel:
    def __init__(self, state: OrderedDict[str, Tensor]):
        tensors = list(state.keys())
        self.layers: List[LayerModel] = []
        i = 0
        while i < len(state):
            self.layers.append(LayerModel(state[tensors[i]], state[tensors[i+1]]))
            i += 2

    def __len__(self):
        return len(self.layers)

    def __iter__(self):
        return iter(self.layers)

    def __getitem__(self, i: int) -> LayerModel:
        return self.layers[i]

nn_model = NNModel(state)

# Converting to a Regular Language

In [ ]:
from abc import ABC, abstractmethod
from typing import List
from difflib import SequenceMatcher

class NNTokensStats:
  def __init__(self, nn_tokens_1: List[str], nn_tokens_2: List[str]):
    self.tok1 = nn_tokens_1
    self.tok2 = nn_tokens_2
    self.longstr1 = "<EOT>".join(nn_tokens_1)
    self.longstr2 = "<EOT>".join(nn_tokens_2)
    
    self.fullTextSimilarity = None
    # self.avgInterSimilarity = None

  def calcStats(self):
    self.fullTextSimilarity = SequenceMatcher(None, self.longstr1, self.longstr2).ratio()
  
  def dumpStats(self):
    print("Comparison stats based on the tokenized NNs:")
    if self.fullTextSimilarity is not None:
      print("Similarity between full texts: " + str(self.fullTextSimilarity))
    print("---------------------------")


class NNTokenizer(ABC):
  @abstractmethod
  def tokenize(self, nnmodel: NNModel) -> List[str]:
    pass
  def compareNNs(self, nnmodel1: NNModel, nnmodel2: NNModel) -> NNTokensStats:
    stats_here = NNTokensStats(self.tokenize(nnmodel1), self.tokenize(nnmodel2))
    stats_here.calcStats()
    stats_here.dumpStats()

class ExampleTokenizer1(NNTokenizer):
  def tokenize(self, nnmodel: NNModel) -> List[str]:
    fullStr = []
    for layer in nnmodel.layers[2:]:
      for neuron in layer.neurons:
        fullStr.append(str(neuron))
    return fullStr

class ExampleTokenizer2(NNTokenizer):
  def tokenize(self, nnmodel: NNModel) -> List[str]:
    fullStr = []
    for layer in nnmodel.layers[2:]:
      for neuron in layer.neurons:
        fullStr.append("another neuron lol")
    return fullStr



285
['l0c256', 'n0b0.010307294316589832c784w-0.038038548082113266w-0.03641679510474205w-0.03551454097032547w-0.03522282466292381w-0.03510757163167w-0.034863777458667755w-0.03478997200727463w-0.03457750007510185w-0.034574955701828w-0.03428623452782631w-0.03428308293223381w-0.034210558980703354w-0.034063976258039474w-0.03391248732805252w-0.03364744037389755w-0.03361698240041733w-0.03329283744096756w-0.03323764353990555w-0.033130355179309845w-0.03312786668539047w-0.03305589780211449w-0.03305394574999809w-0.03300301358103752w-0.032913804054260254w-0.032864004373550415w-0.03285159915685654w-0.03271973505616188w-0.03255535289645195w-0.032528702169656754w-0.03226929157972336w-0.03226041421294212w-0.03224766626954079w-0.032209303230047226w-0.032086990773677826w-0.03206119313836098w-0.031883131712675095w-0.03177376836538315w-0.031671471893787384w-0.031657133251428604w-0.03164633736014366w-0.03163779526948929w-0.031610678881406784w-0.031602829694747925w-0.03154405951499939w-0.03146114945411682w-

In [ ]:
et1 = ExampleTokenizer1()
et2 = ExampleTokenizer2()
tok1 = et1.tokenize(nn_model)
print("finished tok1")
tok2 = et2.tokenize(nn_model)
print("finished tok2")
stats = NNTokensStats(tok1, tok2)
stats.calcStats()
stats.dumpStats()

In [ ]:
# *maybe-possible* but maybe-useless optimization to do at some point:
# all tokenizers are the same, but pass in a func to encode a given neuron-weights-list
# (since that list might be main place where we throw away / select info)

class LineOfStrongestWeightsTokenizer(NNTokenizer):
  def tokenize(self, nnmodel: NNModel) -> List[str]:
    fullStr = []
    for layeridx, layer in enumerate(nnmodel.layers):
      from_neuron = 0
      to_neuron = 0
      strongest_magnitude_weight = 0.0
      thisstr = "L" + str(layeridx) + ":: "
      idx_with_strongest = 0
      for fromneuronidx, fromneuron in enumerate(layer.neurons):
        for toneuronidx, cnxnweight in enumerate(fromneuron.weights):
          if abs(cnxnweight) > abs(strongest_magnitude_weight):
            strongest_magnitude_weight = cnxnweight
            from_neuron = fromneuronidx
            to_neuron = toneuronidx
      thisstr = thisstr + str(from_neuron) + "->" + str(to_neuron) +"as" + str(strongest_magnitude_weight)
      fullStr.append(thisstr)
    return fullStr

losw = LineOfStrongestWeightsTokenizer()
tokmod = losw.tokenize(nn_model)
print(tokmod)


Clearly, this particular tokenizer doesn't hit our compression desiderata:
1. It reveals more-fundamental structure in the NeuralModel data
2. That structure is (ideally) invariant across different NNs that have the same concept (e.g. if two store "addition", we want to inch closer to "both tokenizations contain the same token/token sequence, pointing to their addition connections", while balancing desideratum 3...
3. The tokenization makes searching more intuitive / human-readable, without going to gigantic runtimes.

While these problems won't be solved in our short time allotment, we have a more general system for writing a connection-based tokenization below:

In [ ]:
from typing import Callable

class GeneralTokenizer(NNTokenizer):
  def __init__(self, keepCnxnFunc: Callable):
    self.keepCnxn = keepCnxnFunc

  def tokenize(self, nnmodel: NNModel) -> List[str]:
    fullStr = []
    for layeridx, layer in enumerate(nnmodel.layers):
      layerstring = "L" + str(layeridx) + ": "
      for fromneuronidx, fromneuron in enumerate(layer.neurons):
        numcnxns = len(fromneuron.weights)
        frombias = fromneuron.bias
        for toneuronidx, cnxnweight in enumerate(fromneuron.weights):
          if self.keepCnxn(bias=frombias, cnxns = numcnxns, thisweight = cnxnweight):
            layerstring = layerstring + str(fromneuronidx) + "->" + str(toneuronidx) + ";"
      fullStr.append(layerstring)
    return fullStr

In [ ]:
def ExampleDumbKeepConnection(bias, cnxns, thisweight):
  return thisweight < bias

def ExampleDumbKeepConnection2(bias, cnxns, thisweight):
  return thisweight > bias

newgeneraltokenizer = GeneralTokenizer(ExampleDumbKeepConnection)
tokmod = newgeneraltokenizer.tokenize(nn_model)
print(len("".join(tokmod)))

newgeneraltokenizer2 = GeneralTokenizer(ExampleDumbKeepConnection2)
tokmod2 = newgeneraltokenizer2.tokenize(nn_model)
print(len("".join(tokmod2)))

## Getting all Possible Patterns

So imagine this. We have a neural network that does addition. We want to know whether or not this pattern appears in our neural network. We also don't want to expend the energy of doing subgraph isomorphism right now. Our addition NN is going to contain several layers. We take the first layer of our addition NN and see if it's a subset of any of the other layers. If it's not, we can stop right here, and conclude that no addition is happening, at least not with the algorithm we're considering. Otherwise, we check the next layer of the big NN, and the next layer of the little NN. If all layers match, then it is actually quite feasible that addition is happening in the network. We don't know for sure, because we don't know how the layers actually connect to each other, but now that we know where to look and what to look for, we can use subgraph isomorphism without it being unreasonably inefficient.

In [ ]:
def is_possible_subnetwork(big: NNModel, sub: NNModel) -> bool:
    for i in range(len(big)):
        if big[i].is_superlayer_of(sub[0]):
            is_subnetwork = True
            for j in range(1, len(sub)):
                k = i + j
                if not big[k].is_superlayer_of(sub[j]):
                    is_subnetwork = False
                    break
            if is_subnetwork: return True
    return False

## Future Work

### Optimizing the Equality Check on Neurons

Even on a small neural network, checking for equal neurons is pretty slow. Making this fast would allow this to work on large neural networks. It's not possible with the algorithm we used though. Here's one quick idea for optimizing the `__eq__` method on `NeuronModel`.

```python
def __eq__(self, other: Self):
    if len(self.weights) != len(other.weights): return False
    epsilon = 1 / len(self.weights)
    for i in range(len(self.weights)):
        my_weight = float(self.weights[i])
        other_weight = float(other.weights[i])
        difference = abs(my_weight, other_weight)
        if difference > epsilon: return False
    return True
```

The idea is that if the weights and biases are similar, then the output is probably similar. The problem with this is that there are multiple ways to represent the same neuron.

It also seems that our equivalence function could be highly parallelized. Moving the work over to the GPU may be quite feasible. If that doesn't work, rewriting the project in Rust should be several dozens of times faster.

Another possible improvement is to, instead of having an `excited_states` method, handle the creation of the excited states in the `__eq__` function, and terminate early if something is not found in both. Of course, this only helps if the two neurons are not equal.

### Equivalence of Neurons with More Weights

Not all of the neuron's connections are actually being used. If there are 700 weights in a neuron, it should still be theoretically possible for it to be equivalent to an AND gate.

### Binary Representation of Neural Networks

The text representation of this takes up a lot of space. It shouldn't be too difficult to compress it into a binary format by encoding the string representations of floats into actual binary 32-bit values. This is likely to take up half of the space of the non-binary format.

### Another Approach to Equivalence of Subgraphs

On small subnetworks, we can actually use the same method that was used for finding equivalent neurons to determine if two subnetworks are equivalent. Just try every possible input and see if the output is the same. This obviously isn't possible for the entire graph with 784 inputs, but it might be faster than comparing the individual neurons in some cases, if we limit the size of the input.

### Translation for Transformers

This work has been heavily focused on neural networks, but we expect that some version of this can be done for transformers that are used in language models. We focused on neural networks because that seemed to be the easiest way to get started.